# Balanced buckets

Simulation of the benefit in the number of hop of having balanced DHT buckets.

In [46]:
from binary_trie import Trie, bytes_to_bitstring, int_to_bitstring, bitstring_to_int, bitstring_to_bytes
import multihash as mh
import hashlib as hl
import math, os, random
import numpy as np

## Implementation

System parameters:

In [22]:
# maximal number of buckets in a peer's routing table,
# equivalent to bit size of the keyspace
n_buckets = 256

# length of a kademlia key in bytes
keylen = 32

# bucket containing the closest peer
closest_bucket = 20

# number of peers participating in the network
peers_number = 25000

# ipfs dht concurrency factor, defining how many
# peers are contacted concurrently at each hop
concurrency = 10

# replication factor for all data in the ipfs dht
repl = 20

# size of a bucket in a peer's routing table
bucket_size = 20

In [14]:
# returns a random key in the kademlia keyspace
def random_key():
    return os.urandom(keylen)

In [51]:
def byte_to_int(b: bytes) -> int:
    return int.from_bytes(b, byteorder='big')

In [47]:
def bxor(b1, b2):
    n_b1 = np.frombuffer(b1, dtype='uint8')
    n_b2 = np.frombuffer(b2, dtype='uint8')

    return (n_b1 ^ n_b2).tobytes()

In [48]:
def log_dist(bs0, bs1: bytes) -> int:
    xored = bxor(bs0, bs1)
    dist = byte_to_int(xored)
    if dist == 0:
        return n_buckets
    return n_buckets-math.floor(math.log2(dist))-1


In [49]:
def n_closest(t: Trie, k: bytes, n: int):
    if t.size == 0:
        # TODO: remove me
        print("error nil trie")
        return
    closest = t.n_closest_keys(bytes_to_bitstring(k, l=n_buckets), n)
    return [bitstring_to_bytes(bs) for bs in closest]


In [29]:
# balance_bucket() is given as input a binary trie t, and an integer n,
# it returns a balanced list of n random keys. The list is considered as
# balanced because, if possible it will contain the same number of keys,
# starting with the prefix "0" and "1", the same number of prefixes "00",
# "01", "10", "11" etc.
# this function is recursive
def balance_bucket(t: Trie, n: int):
    if n == 0 or t is None:
        # invalid parameters
        return []

    # number of keys wanted with prefix "0" and "1"
    half = math.ceil(n/2)

    # if at least one branc is undefined, we reached a leaf
    if t.branch[0] is None or t.branch[1] is None:
        # return a list containing the leaf's key
        return [t.key]
    # if both branches have enough candidates
    elif t.branch[0].size >= half and t.branch[1].size >= half:
        add0, add1 = 0, 0
        # odd number of peers to return, need to select a branch
        # to have 1 additional peer
        if n % 2 == 1:
            if random.randint(0, 1) == 0:
                add0 += 1
            else:
                add1 += 1
        # return the union of the n/2 balanced peers starting with
        # prefix "0" and the n/2 starting with prefix "1"
        return balance_bucket(t.branch[0], half-add0) + balance_bucket(t.branch[1], half-add1)
    # less than half keys with prefix "1"
    elif t.branch[0].size >= half:
        b1 = t.branch[1].size
        # return all keys with prefix "1", and a balanced list from
        # bucket 0
        return balance_bucket(t.branch[0], n-b1) + balance_bucket(t.branch[1], b1)
    # less than half keys with prefix "0"
    elif t.branch[1].size >= half:
        b0 = t.branch[0].size
        # return all keys with prefix "0", and a balanced list from
        # with prefix "1"
        return balance_bucket(t.branch[0], b0) + balance_bucket(t.branch[1], n-b0)
    else:
        # there are less than n keys in t, return all keys
        return t.match_prefix_keys(prefix="")

In [97]:
# binary trie storing all peers ids
sim_trie = Trie()
# list of peers for this simulation
sim_peers = []

# routing table of the simluation with random buckets
sim_peers_random = {}
# routing table of the simulation with balanced buckets
sim_peers_balanced = {}

# generate random peerids and populate the binary trie
for _ in range(peers_number):
    # generate random peerid
    peerid = random_key()
    # add peerid to peer trie
    while not sim_trie.add(bytes_to_bitstring(peerid)):
        # add returns false if key already in trie
        # generate new identity
        peerid = random_key()

    # add peerid to list of peerids
    sim_peers.append(peerid)

    # create entry for peerid in both simulations routing tables
    sim_peers_random[peerid] = {}
    sim_peers_balanced[peerid] = {}

# iterate on all possible buckets, large to small buckets, 0 prefix to 1 prefix
for bid in range(closest_bucket):
    for i in range(2**(bid+1)):
        # prefix of the current bucket
        bucket_prefix = int_to_bitstring(i, bid+1)
        # all peers falling in this bucket
        candidates = sim_trie.match_prefix_keys(bucket_prefix)

        # if no candidates, continue
        if len(candidates) == 0:
            continue

        # flip last bit
        target_prefix = bucket_prefix[:-1]+str(1-int(bucket_prefix[-1]))

        # targets are peers that could use the candidates (of the current bucket)
        # in their routing table
        targets = sim_trie.match_prefix_keys(target_prefix)


        # bucket root is the subtrie representing the bucket
        bucket_root = sim_trie.find_trie(bucket_prefix)
        if bucket_root is None:
            # bucket root could be undefined if subkey not in the trie
            # we need to find the closest child
            i = 0
            while bucket_root is None:
                i += 1
                # find the first existing parent
                bucket_root = sim_trie.find_trie(bucket_prefix[:-i])
            # select the appropriate child as the bucket root
            bucket_root = bucket_root.branch[int(bucket_prefix[-i+1])]

        # iterate on all targets, and define which candidates are selected
        # for their bucket
        for t in targets:
            # if more than `bucket_size` candidates, select `bucket_size` of them
            # at random, else take them all
            random_bucket = [bitstring_to_bytes(p) for p in 
                random.sample(candidates, min(bucket_size, len(candidates)))]
            # add these peers to t's routing table, in bucket bid
            sim_peers_random[bitstring_to_bytes(t)][bid] = random_bucket

            # randomly select candidates in a balanced manner
            balanced_bucket = [bitstring_to_bytes(p) for p in balance_bucket(bucket_root, min(bucket_size, len(candidates)))]
            # add these peers to t's routing table, in bucket bid
            sim_peers_balanced[bitstring_to_bytes(t)][bid] = balanced_bucket


58.4 s for bucket_size = 20


In [98]:
def lookup_n_hops(peer, key, peers, trie):
    # closest peers are the peers storing data associated with key
    closest_peers = n_closest(trie, key, repl)

    # hop count
    hop = 0
    if peer in closest_peers:
        # if peer store is among the closest peers, 0 hop
        return hop

    # binary trie helping find the closest peers to key in 
    # the node's routing table
    req_trie = Trie()
    # bucket id, corresponding to key
    bid = log_dist(peer, key)

    if bid in peers[peer] and len(peers[peer][bid]) > 0:
        # if this bucket is populated, add peers to req_trie
        for peer_neighbor in peers[peer][bid]:
            req_trie.add(bytes_to_bitstring(peer_neighbor, l=n_buckets))
    else:
        buckets = list(peers[peer].keys())
        found_higher = False
        for b in range(bid+1, max(buckets)+1):
            if b in buckets and len(peers[peer][b]) > 0:
                for peer_neighbor in peers[peer][b]:
                    req_trie.add(bytes_to_bitstring(peer_neighbor, l=n_buckets))
                    found_higher = True
        if not found_higher:
            for b in range(bid-1, -1, -1):
                if b in buckets and len(peers[peer][b]) > 0:
                    for peer_neighbor in peers[peer][b]:
                        req_trie.add(bytes_to_bitstring(peer_neighbor, l=n_buckets))
                    break # we are only interested in the closest bucket
    if req_trie.size == 0:
        print(peers[peer])
        print("empty trie")
    req_peers = n_closest(req_trie, key, concurrency)

    while True:
        hop += 1
        for c in closest_peers:
            if c in req_peers:
                return hop

        req_trie = Trie()
        for p in req_peers:
            bid = log_dist(p, key)

            if bid in peers[p] and len(peers[p][bid]) > 0:
                for peer_neighbor in peers[p][bid]:
                    req_trie.add(bytes_to_bitstring(peer_neighbor, l=n_buckets))
            else:
                buckets = list(peers[p].keys())
                found_higher = False
                for b in range(bid+1, max(buckets)+1):
                    if b in buckets and len(peers[p][b]) > 0:
                        for peer_neighbor in peers[p][b]:
                            req_trie.add(bytes_to_bitstring(peer_neighbor, l=n_buckets))
                            found_higher = True
                if not found_higher:
                    for b in range(bid-1, -1, -1):
                        if b in buckets and len(peers[p][b]) > 0:
                            for peer_neighbor in peers[p][b]:
                                req_trie.add(bytes_to_bitstring(peer_neighbor, l=n_buckets))
                            break # we are only interested in the closest bucket
                    
        req_peers = n_closest(req_trie, key, 20)

# Running experiments

In [99]:
sim_random_hop_count = []
for i in range(10000):
    peer = random.choice(sim_peers)
    key = random_key()
    n_hop = lookup_n_hops(peer, key, sim_peers_random, sim_trie)
    sim_random_hop_count.append(n_hop)


In [100]:
sim_avg_random = sum(sim_random_hop_count)/len(sim_random_hop_count)
sim_avg_random

1.9227

In [101]:
sim_balanced_hop_count = []
for i in range(10000):
    peer = random.choice(sim_peers)
    key = random_key()
    n_hop = lookup_n_hops(peer, key, sim_peers_balanced, sim_trie)
    sim_balanced_hop_count.append(n_hop)

In [102]:
sim_avg_balanced = sum(sim_balanced_hop_count)/len(sim_balanced_hop_count)
sim_avg_balanced

1.8973

In [103]:
print("Average random buckets", sim_avg_random)
print("Average balanced buckets", sim_avg_balanced)
print(1-sim_avg_random/sim_avg_balanced)

Average random buckets 1.9227
Average balanced buckets 1.8973
-0.013387445317029512
